In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets,transforms,models
import os
import shutil
import time
from torch.autograd import Variable
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import cv2

In [ ]:
# 将训练集划分出10%的数据量作为验证集
# 把训练集中的猫狗图像分别放在独立的文件夹中
# 文件结构为：
train
copy
    -train
        -cats
        -dogs
    -val
        -cats
        -dogs
test

In [2]:
original_dataset_dir='./train'
total_num=int(len(os.listdir(original_dataset_dir))/2)
random_idx=np.array(range(total_num))
np.random.shuffle(random_idx)

In [3]:

base_dir='./copy'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
sub_dirs=['train','val']
animals=['cats','dogs']
train_idx=random_idx[:int(total_num*0.9)]
val_idx=random_idx[int(total_num*0.9):]
print(train_idx)
print(val_idx)
print(len(train_idx))
print(len(val_idx))

[10584  9517 10546 ... 10179  3482 11039]
[ 7276 11428 11791 ... 10335   920  5268]
11250
1250


In [6]:
numbers=[train_idx,val_idx]
for idx,sub_dir in enumerate(sub_dirs):
    dir=os.path.join(base_dir,sub_dir)
    if not os.path.exists(dir):
        os.mkdir(dir)
    for animal in animals:
        animal_dir=os.path.join(dir,animal)
        if not os.path.exists(animal_dir):
            os.mkdir(animal_dir)
        fnames=[animal[:-1]+'.{}.jpg'.format(i) for i in numbers[idx]]
        for fname in fnames:
            src=os.path.join(original_dataset_dir,fname)
            dst=os.path.join(animal_dir,fname)
            shutil.copyfile(src,dst)
        
        print(animal_dir+' total images : %d' % (len(os.listdir(animal_dir))))

./copy\train\cats total images : 11250
./copy\train\dogs total images : 11250
./copy\val\cats total images : 1250
./copy\val\dogs total images : 1250


In [ ]:
# 计算训练集的均值和偏差

In [7]:
img_filenames=os.listdir('./train/')
m_list,s_list=[],[]
for img_filename in tqdm(img_filenames):
    img=cv2.imread('./train/'+img_filename)#BGR
    img=img/255.0
    m,s=cv2.meanStdDev(img)
    m_list.append(m.reshape((3,)))
    s_list.append(s.reshape((3,)))
m_array=np.array(m_list)
s_array=np.array(s_list)
m=m_array.mean(axis=0,keepdims=True)#BGR
s=s_array.mean(axis=0,keepdims=True)
print(m[0][::-1])#RGB
print(s[0][::-1])

100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [02:07<00:00, 195.32it/s]


[0.48832284 0.45508163 0.41695606]
[0.22944327 0.22498401 0.22518628]
